# 画像処理編 - 17
**2値画像をZhang Suenのアルゴリズムにより細線化するプログラム**

## 必要なライブラリを読み込む
OpenCVでZhang-Suenの細線化アルゴリズムを利用するには，`opencv-contrib-python`をインストールする必要がある．<br>
OpenCVには4つの異なるパッケージがあるが，複数のOpenCVパッケージを共存させることは不可能なので注意．
- `opencv-python`：安定版
- `opencv-contrib-python`：OpenCVの拡張モジュール群（開発版）
- `opencv-python-headless`：GUI機能の無いOpenCVの安定版
- `opencv-contrib-python-headless`：GUI機能の無いOpenCVの開発版

In [144]:
import cv2

## 画像を読み込む

In [145]:
img = cv2.cvtColor(cv2.imread("../../data/kanji.png"), cv2.COLOR_BGR2GRAY)

## Zhang-Suenの細線化アルゴリズムによる細線化

### Zhang-Suenの細線化アルゴリズムの仕組み
Zhang-Suenの細線化アルゴリズムでは，以下で説明する2つのステップを画像に繰り返し適用することで，2値画像を細線化することができる．<br>
#### 前提
注目する画素をP1とし，その近傍の画素を下のように定める．
$$
\begin{matrix}
P9 & P2 & P3 \\ P8 & P1 & P4 \\ P7 & P6 & P5
\end{matrix}
$$
また，以下の2つの関数を定める．<br>
$N(P1)$：非ゼロの$P1$の近傍の数を返す関数<br>
$S(P1)$：$P2,P3,\cdots,P8,P9,P2$と時計回りに見たときに，$0\rightarrow 1$の遷移の数を返す関数<br>
Zhang-Suenの細線化アルゴリズムで扱う2値画像は，白黒を反転させた2値画像である必要がある．ただし，白を0，黒を1とする．
#### ステップ1
全ての画素を検証し，以下の条件を同時に全て満たす画素を記録しておく．<br>
1. $P1=1$で，8個の近傍を持つ
2. $2\le N(P1)\le6$
3. $S(P1)=1$
4. $P2\times P4\times P6=0$
5. $P4\times P6\times P8=0$

画像上を走査後，記録された画素を全て0に設定する．
#### ステップ2
全ての画素を検証し，以下の条件を同時に全て満たす画素を記録しておく．<br>
1. $P1=1$で，8個の近傍を持つ
2. $2\le N(P1)\le6$
3. $S(P1)=1$
4. $P2\times P4\times P8=0$
5. $P2\times P6\times P8=0$

画像上を走査後，記録された画素を全て0に設定する．
#### ステップ1とステップ2を繰り返す
ステップ1とステップ2を画像に繰り返し適用し，画素値の変化が無くなればアルゴリズムを停止する．

### 前処理
1. ガウシアンフィルタによる画像の平滑化（この後の処理へのノイズの影響を軽減するため）
2. 大津の方法による画像の二値化（白黒反転した二値化画像を得る必要あり）

In [ ]:
# ノイズの影響を軽減するために，ガウシアンフィルタで平滑化
blur_img = cv2.GaussianBlur(img, (5, 5), 3)

# グレースケール画像の二値化（大津の方法）
# THRESH_OTSUは閾値を決定するだけ，二値化のためには他のフラグと組み合わせる必要がある（THRESH_BINARY, THRESH_BINARY_INV）
# Zhang-Suenの細線化アルゴリズムを使うには，白黒を反転させる必要があるので，THRESH_BINARY_INV
th_val, bin_img = cv2.threshold(blur_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

### 細線化処理
`cv2.ximgproc`（Extended Image Processing）パッケージの`thinning()`関数を用いる．
```python
dst = cv2.ximproc.thinning(src: Mat, thinningType: int = ...)
```
- `src`：入力する2値画像<br>
- `dst`：出力画像<br>
- `thinningType`：使用する細線化アルゴリズム（初期値は`cv2.ximgproc.THINNING_ZHANGSUEN`）

In [146]:
# 細線化アルゴリズムの適用
img_thinned = cv2.ximgproc.thinning(bin_img, cv2.ximgproc.THINNING_ZHANGSUEN)

th_val, img_thinned = cv2.threshold(img_thinned, 0, 255, cv2.THRESH_BINARY_INV) # 反転していた白黒を元に戻す

## 処理済み画像の出力

In [147]:
cv2.imwrite("../../data/jupyter-notebook/ip_17.jpg", img_thinned)

True

## 原画像と細線化した画像の比較
<div align=center>
<img src="./img/kanji.png">
<img src="./img/ip_17.jpg">
</div>